In [ ]:
# Load dependencies
import pandas as pd
import time
from nba_api.stats.endpoints import commonteamroster

In [ ]:
# Load the list of team IDs
teams = pd.read_csv("nba_teams.csv")

# CommonTeamRoster Parameters
season = "2025-26"
league_id = "00"

# Initialize an empty list to store each team's roster DataFrame
all_rosters = []

# Loop through each team ID and get the roster
for i in teams["id"]:
    try:
        print(f"Fetching roster for team ID {i}...")
        roster = commonteamroster.CommonTeamRoster(
            team_id=i,
            season=season,
            league_id_nullable=league_id,
            timeout=60
        )
        roster_df = roster.get_data_frames()[0]
        all_rosters.append(roster_df)
        
        # Delay between teams to avoid rate-limiting
        time.sleep(3)
        
    except Exception as e:
        print(f"Failed for team ID {i}: {e}")

# Concatenate all the roster DataFrames into one DataFrame
full_roster_df = pd.concat(all_rosters, ignore_index=True)

full_roster_df.head()


Fetching roster for team ID 1610612737...
Fetching roster for team ID 1610612738...
Fetching roster for team ID 1610612739...
Fetching roster for team ID 1610612740...
Fetching roster for team ID 1610612741...
Fetching roster for team ID 1610612742...
Fetching roster for team ID 1610612743...
Fetching roster for team ID 1610612744...
Fetching roster for team ID 1610612745...
Fetching roster for team ID 1610612746...
Fetching roster for team ID 1610612747...
Fetching roster for team ID 1610612748...
Fetching roster for team ID 1610612749...
Fetching roster for team ID 1610612750...
Fetching roster for team ID 1610612751...
Fetching roster for team ID 1610612752...
Fetching roster for team ID 1610612753...
Fetching roster for team ID 1610612754...
Fetching roster for team ID 1610612755...
Fetching roster for team ID 1610612756...
Fetching roster for team ID 1610612757...
Fetching roster for team ID 1610612758...
Fetching roster for team ID 1610612759...
Fetching roster for team ID 161061

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612737,2025,00,Nikola Đurišić,Nikola,nikola-đurišić,None,G,6-8,214,"FEB 23, 2004",21.0,R,Mega Basket,1642365,Draft Rights Traded from MIA on 06/27/24
1,1610612737,2025,00,Eli John Ndiaye,Eli John,eli-john-ndiaye,None,F,6-8,209,"JUN 26, 2004",21.0,R,Real Madrid,1642947,Signed on 07/03/25
2,1610612737,2025,00,Jacob Toppin,Jacob,jacob-toppin,00,F,6-8,200,"MAY 08, 2000",25.0,2,Kentucky,1631210,Signed on 03/04/25
3,1610612737,2025,00,Jalen Johnson,Jalen,jalen-johnson,1,F,6-8,219,"DEC 18, 2001",23.0,4,Duke,1630552,#20 Pick in 2021 Draft
4,1610612737,2025,00,Keaton Wallace,Keaton,keaton-wallace,2,G,6-3,185,"FEB 26, 1999",26.0,1,Texas-San Antonio,1630811,Signed on 09/29/23


In [6]:
full_roster_with_team_name = full_roster_df.merge(
    teams[['id', 'full_name']], 
    left_on='TeamID', 
    right_on='id', 
    how='left'
)

# Concatenate all the roster DataFrames into one DataFrame
full_roster_df = pd.concat(all_rosters, ignore_index=True)

full_roster_with_team_name = full_roster_df.merge(
    teams[['id', 'full_name']], 
    left_on='TeamID', 
    right_on='id', 
    how='left'
)

# Drop the 'id' column from the merged DataFrame
full_roster_with_team_name = full_roster_with_team_name.drop(columns=['id'])
full_roster_with_team_name = full_roster_with_team_name.rename(columns={'full_name': 'TEAM_NAME'})

# Re-order columns
new_column_order = ['TeamID', 'TEAM_NAME', 'PLAYER_ID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG',
       'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP',
       'SCHOOL', 'SEASON', 'LeagueID', 'HOW_ACQUIRED']

player_df = full_roster_with_team_name[new_column_order]

# Display the combined DataFrame
player_df.sample(10)

,TeamID,TEAM_NAME,PLAYER_ID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,SEASON,LeagueID,HOW_ACQUIRED
292,1610612751,Brooklyn Nets,1630560,Cam Thomas,Cam,cam-thomas,24,G,6-3,210,"OCT 13, 2001",23.0,4,Louisiana State,2025,00,#27 Pick in 2021 Draft
519,1610612762,Utah Jazz,1641729,Brice Sensabaugh,Brice,brice-sensabaugh,28,F,6-5,235,"OCT 30, 2003",21.0,2,Ohio State,2025,00,#28 Pick in 2023 Draft
57,1610612740,New Orleans Pelicans,1631232,Keion Brooks Jr.,Keion,keion-brooks-jr,None,F,6-7,210,"AUG 07, 2000",24.0,1,Washington,2025,00,Signed on 09/23/24
272,1610612750,Minnesota Timberwolves,1642866,Joan Beringer,Joan,joan-beringer,19,F,6-10,230,"NOV 11, 2006",18.0,R,Cedevita Olimpija,2025,00,#17 Pick in 2025 Draft
359,1610612755,Philadelphia 76ers,1641780,Johni Broome,Johni,johni-broome,None,F,6-10,235,"JUL 19, 2002",23.0,R,Auburn,2025,00,#35 Pick in 2025 Draft
569,1610612765,Detroit Pistons,1641709,Ausar Thompson,Ausar,ausar-thompson,9,G-F,6-6,205,"JAN 30, 2003",22.0,2,Overtime Elite,2025,00,#5 Pick in 2023 Draft
388,1610612756,Phoenix Suns,1631109,Mark Williams,Mark,mark-williams,5,C,7-0,240,"DEC 16, 2001",23.0,3,Duke,2025,00,Traded from CHA on 06/30/25
448,1610612759,San Antonio Spurs,1630561,David Duke Jr.,David,david-duke-jr,7,G,6-4,204,"OCT 13, 1999",25.0,4,Providence,2025,00,Signed on 12/14/23
80,1610612741,Chicago Bulls,1642934,Caleb Grill,Caleb,caleb-grill,None,G,6-3,205,"JUN 15, 2000",25.0,R,Missouri,2025,00,Signed on 07/06/25
570,1610612765,Detroit Pistons,202699,Tobias Harris,Tobias,tobias-harris,12,F,6-8,226,"JUL 15, 1992",33.0,14,Tennessee,2025,00,Signed on 07/08/24


In [7]:
# Export player list to CSV
player_df.to_csv("nba_players.csv", index = False)